# Quant Strategy Packaging Demo

This notebook demonstrates how to package a trading strategy developed in a Jupyter notebook into a deployable Docker container.

## Workflow Overview

1. Develop and test your strategy
2. Package the strategy using `StrategyPackager`
3. Test the packaged strategy locally
4. Build a Docker deployment
5. Deploy and use the API

In [ ]:
# Install the library (if not already installed)
# !pip install -e ..

import pandas as pd
import numpy as np
from datetime import datetime, timedelta

## Step 1: Create Sample Market Data

Let's create some sample OHLCV data for testing.

In [ ]:
# Generate sample market data
np.random.seed(42)
dates = pd.date_range(start='2024-01-01', end='2024-06-01', freq='D')
n = len(dates)

# Simulate price data with some trend and noise
base_price = 100
trend = np.linspace(0, 20, n)
noise = np.random.randn(n) * 2
close_prices = base_price + trend + noise

data = pd.DataFrame({
    'open': close_prices + np.random.randn(n) * 0.5,
    'high': close_prices + np.abs(np.random.randn(n)) * 1.5,
    'low': close_prices - np.abs(np.random.randn(n)) * 1.5,
    'close': close_prices,
    'volume': np.random.randint(1000000, 5000000, n)
}, index=dates)

print("Sample market data:")
print(data.head())
print(f"\nShape: {data.shape}")

## Step 2: Define Your Trading Strategy

Here's a simple momentum strategy that:
- Goes long when price is above the 20-day moving average
- Goes flat when price is below the moving average

This follows the pattern expected by the quant-helper library:
- Input: DataFrame with OHLCV data
- Output: Series with signals (1.0=long, 0.0=flat, -1.0=short)

In [ ]:
def momentum_strategy(data):
    """
    Simple momentum strategy based on moving average crossover.
    
    Args:
        data: DataFrame with OHLCV columns
        
    Returns:
        Series with position signals (1.0=long, 0.0=flat, -1.0=short)
    """
    # Calculate 20-day moving average
    ma_20 = data['close'].rolling(window=20).mean()
    
    # Generate signals
    signals = pd.Series(0.0, index=data.index)
    signals[data['close'] > ma_20] = 1.0  # Long when price above MA
    signals[data['close'] <= ma_20] = 0.0  # Flat when price below MA
    
    return signals

# Test the strategy
signals = momentum_strategy(data)
print("Strategy signals:")
print(signals.tail(10))
print(f"\nLong positions: {(signals == 1.0).sum()}")
print(f"Flat positions: {(signals == 0.0).sum()}")

## Step 3: Package the Strategy

Now let's package this strategy so it can be deployed.

In [ ]:
from quant_packaging import StrategyPackager

# Create packager
packager = StrategyPackager(output_dir="./strategies")

# Save the strategy
strategy_dir = packager.save_strategy(
    strategy_func=momentum_strategy,
    name="momentum_ma20",
    description="Momentum strategy using 20-day moving average crossover",
    requirements=["pandas>=1.5.0", "numpy>=1.24.0"],
    version="1.0.0",
    metadata={
        "author": "Demo User",
        "strategy_type": "momentum",
        "timeframe": "daily"
    }
)

print(f"\nStrategy packaged successfully!")

## Step 4: Test the Packaged Strategy

Before deploying, let's verify the packaged strategy works correctly.

In [ ]:
from quant_packaging import StrategyContainer

# Load the packaged strategy
container = StrategyContainer("./strategies/momentum_ma20")

# Get info
print("Strategy Info:")
print(container)
info = container.get_info()
print(f"\nMetadata: {info['metadata']['description']}")
print(f"Version: {info['metadata']['version']}")

# Run the strategy on test data
test_signals = container.run(data)

# Verify results match original
print(f"\nVerification:")
print(f"Original signals: {signals.sum()}")
print(f"Loaded signals: {test_signals.sum()}")
print(f"Match: {np.allclose(signals, test_signals, equal_nan=True)}")

## Step 5: Build Docker Deployment

Now let's create a Docker deployment with a REST API.

In [ ]:
from quant_packaging import DockerBuilder

# Create Docker deployment
builder = DockerBuilder(output_dir="./deployments")

deployment_dir = builder.create_deployment(
    strategy_name="momentum_ma20",
    strategy_dir="./strategies/momentum_ma20",
    port=8000,
    python_version="3.11"
)

print(f"\nDeployment created at: {deployment_dir}")
print("\nTo deploy:")
print(f"  cd {deployment_dir}")
print(f"  docker-compose up --build")

## Step 6: List All Strategies

You can list all packaged strategies.

In [ ]:
# List all strategies
strategies = packager.list_strategies()

print(f"Found {len(strategies)} strategy(ies):\n")
for strat in strategies:
    print(f"Name: {strat['name']}")
    print(f"  Version: {strat['version']}")
    print(f"  Description: {strat['description']}")
    print(f"  Created: {strat['created_at']}")
    print()

## Step 7: Test the API (After Deployment)

Once you've deployed the Docker container, you can test the API like this:

In [ ]:
# This cell assumes the Docker container is running
# Run: cd deployments/momentum_ma20 && docker-compose up

import requests

API_URL = "http://localhost:8000"

try:
    # Check health
    response = requests.get(f"{API_URL}/health")
    print("Health check:", response.json())
    
    # Get info
    response = requests.get(f"{API_URL}/info")
    print("\nStrategy info:", response.json())
    
    # Make prediction
    test_data = data.tail(50).reset_index()
    test_data['timestamp'] = test_data['index'].astype(str)
    test_data = test_data.drop('index', axis=1)
    
    payload = {
        "data": test_data.to_dict(orient='records')
    }
    
    response = requests.post(f"{API_URL}/predict", json=payload)
    result = response.json()
    
    print(f"\nReceived {len(result['signals'])} signals")
    print(f"Sample signals: {result['signals'][:5]}")
    
except requests.exceptions.ConnectionError:
    print("⚠️  Docker container not running. Deploy first with:")
    print("   cd deployments/momentum_ma20 && docker-compose up")

## Next Steps

Now you have a complete workflow for packaging and deploying trading strategies!

### Advanced Examples

Try creating more sophisticated strategies:

1. **Mean Reversion Strategy**
```python
def mean_reversion_strategy(data):
    ma = data['close'].rolling(20).mean()
    std = data['close'].rolling(20).std()
    
    signals = pd.Series(0.0, index=data.index)
    signals[data['close'] < ma - 2*std] = 1.0  # Buy when oversold
    signals[data['close'] > ma + 2*std] = -1.0  # Sell when overbought
    
    return signals
```

2. **Multi-Timeframe Strategy**
```python
def multi_tf_strategy(data):
    fast_ma = data['close'].rolling(10).mean()
    slow_ma = data['close'].rolling(50).mean()
    
    signals = pd.Series(0.0, index=data.index)
    signals[(fast_ma > slow_ma)] = 1.0
    signals[(fast_ma <= slow_ma)] = 0.0
    
    return signals
```

### Deployment Options

- **Local Docker**: `docker-compose up`
- **Cloud**: Push to Docker Hub and deploy to AWS ECS, Google Cloud Run, etc.
- **Kubernetes**: Use the generated Dockerfile with your k8s manifests

### API Documentation

Visit `http://localhost:8000/docs` when your container is running to see the interactive API documentation (Swagger UI).